# Load Dataset

In [3]:
import pandas as pd

# Load your raw dataset
df = pd.read_excel("..\data\TextDataSet.xlsx")

print("✅ Dataset loaded successfully!")
print(df.shape)
print(df.columns)
df.head()


✅ Dataset loaded successfully!
(363, 4)
Index(['Plant Name', 'Disease Name', 'Symptoms', 'Treatments'], dtype='object')


,Plant Name,Disease Name,Symptoms,Treatments
0,Tomato,Early Blight,Brown concentric spots (target spots) on older...,Apply Mancozeb or Chlorothalonil; remove infec...
1,Tomato,Late Blight,Dark patches on leaves and fruits; foul odor; ...,Spray Metalaxyl or Copper oxychloride; improve...
2,Tomato,Leaf Curl Virus,Upward curling leaves; stunted plant; reduced ...,Control whiteflies (vector) with insecticides;...
3,Tomato,Bacterial Canker,White-margined spots (bird's-eye) on fruit; ma...,Use clean seeds; apply Copper spray; avoid spl...
4,Tomato,Target Spot,"Small, dark brown to black spots with light ce...",Apply Chlorothalonil or Mancozeb; ensure good ...


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Cleans, tokenizes, removes stopwords, and lemmatizes text."""
    if pd.isnull(text):
        return ""
    text = text.lower()  # lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    words = nltk.word_tokenize(text)  # tokenize
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]  # remove stopwords & lemmatize
    return " ".join(words)

# Apply to all text columns
for col in ["Plant Name", "Disease Name", "Symptoms", "Treatments"]:
    new_col = "Cleaned_" + col.replace(" ", "_")
    df[new_col] = df[col].apply(preprocess_text)

print("✅ All text columns cleaned successfully!")
df.head()


[nltk_data] Error loading punkt: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>
[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>
[nltk_data] Error loading wordnet: <urlopen error Tunnel connection
[nltk_data]     failed: 407 Proxy Authentication Required>


✅ All text columns cleaned successfully!


,Plant Name,Disease Name,Symptoms,Treatments,Cleaned_Plant_Name,Cleaned_Disease_Name,Cleaned_Symptoms,Cleaned_Treatments
0,Tomato,Early Blight,Brown concentric spots (target spots) on older...,Apply Mancozeb or Chlorothalonil; remove infec...,tomato,early blight,brown concentric spot target spot older leaf y...,apply mancozeb chlorothalonil remove infected ...
1,Tomato,Late Blight,Dark patches on leaves and fruits; foul odor; ...,Spray Metalaxyl or Copper oxychloride; improve...,tomato,late blight,dark patch leaf fruit foul odor rapid blighting,spray metalaxyl copper oxychloride improve air...
2,Tomato,Leaf Curl Virus,Upward curling leaves; stunted plant; reduced ...,Control whiteflies (vector) with insecticides;...,tomato,leaf curl virus,upward curling leaf stunted plant reduced frui...,control whitefly vector insecticide use resist...
3,Tomato,Bacterial Canker,White-margined spots (bird's-eye) on fruit; ma...,Use clean seeds; apply Copper spray; avoid spl...,tomato,bacterial canker,whitemargined spot birdseye fruit marginal lea...,use clean seed apply copper spray avoid splash...
4,Tomato,Target Spot,"Small, dark brown to black spots with light ce...",Apply Chlorothalonil or Mancozeb; ensure good ...,tomato,target spot,small dark brown black spot light center leaf ...,apply chlorothalonil mancozeb ensure good air ...


# Balance the Dataset

In [24]:
from sklearn.utils import resample

# Create a balanced dataset based on 'Cleaned_Disease_Name'
balanced_df = pd.DataFrame()

# Find minimum and maximum class counts
class_counts = df['Cleaned_Disease_Name'].value_counts()
min_class = class_counts.min()
max_class = class_counts.max()

print(f"📊 Before balancing: {len(df)} samples, {len(class_counts)} unique diseases")

# Oversample rare classes up to a fixed limit
target_size = min(max_class, 30)  # cap at 30 per disease to avoid overfitting

for disease, subset in df.groupby('Cleaned_Disease_Name'):
    if len(subset) < target_size:
        subset_resampled = resample(subset, 
                                    replace=True, 
                                    n_samples=target_size, 
                                    random_state=42)
    else:
        subset_resampled = subset.sample(target_size, random_state=42)
    balanced_df = pd.concat([balanced_df, subset_resampled])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

print("✅ Balancing complete!")
print("📈 After balancing:", len(balanced_df), "samples")
print("🧩 Each disease count:")
print(balanced_df['Cleaned_Disease_Name'].value_counts().head(10))


📊 Before balancing: 363 samples, 243 unique diseases
✅ Balancing complete!
📈 After balancing: 3645 samples
🧩 Each disease count:
Cleaned_Disease_Name
black knot dibotryon             15
powdery mildew                   15
fruit shoot borer                15
gray leaf spot pestalotiopsis    15
leaf spot                        15
white rust                       15
root rot aphanomyces             15
fruit spot cercospora            15
rhizosphaera needle cast         15
fruit canker                     15
Name: count, dtype: int64


In [25]:
import random

# Function to generate natural symptom sentences
def paraphrase(text):
    templates = [
        f"Symptoms observed: {text}",
        f"The plant exhibits {text}.",
        f"Common signs include {text}.",
        f"Leaves show {text}.",
        f"Typical symptom: {text}.",
        f"This disease shows {text} on affected parts."
    ]
    return random.choice(templates)

# Apply paraphrasing to the cleaned symptom column
balanced_df["Expanded_Symptoms"] = balanced_df["Cleaned_Symptoms"].apply(paraphrase)

print("✅ Paraphrased symptom sentences created successfully!")
print(balanced_df[["Cleaned_Symptoms", "Expanded_Symptoms"]].head(10))


✅ Paraphrased symptom sentences created successfully!
                                    Cleaned_Symptoms  \
0         hard black warty swelling knot twig branch   
1           small circular dark spot fruit leaf twig   
2                     reddishpurple spot flower leaf   
3  sudden irreversible wilting without leaf yello...   
4                         curled leaf mosaic pattern   
5             concentric dark spot light center leaf   
6               dark irregular lesion leaflet margin   
7            salmonpink fungal spore dying leaf stem   
8               yellow spot needle turn reddishbrown   
9    yellow brown spot leaf turning rusty underneath   

                                   Expanded_Symptoms  
0  Typical symptom: hard black warty swelling kno...  
1  The plant exhibits small circular dark spot fr...  
2  This disease shows reddishpurple spot flower l...  
3  This disease shows sudden irreversible wilting...  
4  This disease shows curled leaf mosaic pattern ...  

# Final Clean-Up of Expanded Text

In [26]:
import re

def final_clean(text):
    if pd.isnull(text):
        return ""
    text = text.lower()  # convert to lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation/symbols
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    return text

balanced_df["Final_Symptoms"] = balanced_df["Expanded_Symptoms"].apply(final_clean)

print("✅ Final cleanup done on expanded symptoms!")
print(balanced_df[["Expanded_Symptoms", "Final_Symptoms"]].head(10))


✅ Final cleanup done on expanded symptoms!
                                   Expanded_Symptoms  \
0  Typical symptom: hard black warty swelling kno...   
1  The plant exhibits small circular dark spot fr...   
2  This disease shows reddishpurple spot flower l...   
3  This disease shows sudden irreversible wilting...   
4  This disease shows curled leaf mosaic pattern ...   
5  Typical symptom: concentric dark spot light ce...   
6  Leaves show dark irregular lesion leaflet margin.   
7  The plant exhibits salmonpink fungal spore dyi...   
8  Leaves show yellow spot needle turn reddishbrown.   
9  Typical symptom: yellow brown spot leaf turnin...   

                                      Final_Symptoms  
0  typical symptom hard black warty swelling knot...  
1  the plant exhibits small circular dark spot fr...  
2  this disease shows reddishpurple spot flower l...  
3  this disease shows sudden irreversible wilting...  
4  this disease shows curled leaf mosaic pattern ...  
5  typical

In [27]:
# Save the fully cleaned, balanced, and finalized dataset
balanced_df.to_excel("..\data\processed\Final_Cleaned_Balanced_Dataset.xlsx", index=False)

print("✅ Final dataset saved successfully!")
print("📁 Saved as: Final_Cleaned_Balanced_Dataset.xlsx in your PlantDocBot folder")
print("📊 Shape:", balanced_df.shape)
print("🧩 Columns:", list(balanced_df.columns))


✅ Final dataset saved successfully!
📁 Saved as: Final_Cleaned_Balanced_Dataset.xlsx in your PlantDocBot folder
📊 Shape: (3645, 10)
🧩 Columns: ['Plant Name', 'Disease Name', 'Symptoms', 'Treatments', 'Cleaned_Plant_Name', 'Cleaned_Disease_Name', 'Cleaned_Symptoms', 'Cleaned_Treatments', 'Expanded_Symptoms', 'Final_Symptoms']


In [5]:
import pandas as pd

# Load your last cleaned and expanded dataset
df = pd.read_excel("..\data\processed\Final_Cleaned_Balanced_Dataset.xlsx")

print("✅ Dataset loaded successfully!")
print(df.head())


✅ Dataset loaded successfully!
  Plant Name            Disease Name  \
0       Plum  Black Knot (Dibotryon)   
1      Peach     Scab (Cladosporium)   
2    Dogwood        Spot Anthracnose   
3      Chili          Bacterial Wilt   
4     Papaya         Leaf Curl Virus   

                                            Symptoms  \
0  Hard, black, warty swellings (knots) on twigs ...   
1  Small, circular, dark spots on fruit, leaves, ...   
2         Reddish-purple spots on flowers and leaves   
3  Sudden, irreversible wilting without leaf yell...   
4                     Curled leaves; mosaic patterns   

                                          Treatments Cleaned_Plant_Name  \
0  Prune out knots in winter; apply Chlorothaloni...               plum   
1       Apply Captan or Chlorothalonil at petal fall              peach   
2                  Apply Chlorothalonil at bud break            dogwood   
3  Plant resistant varieties; avoid planting in i...              chili   
4      Control w

# Label Encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["Cleaned_Disease_Name"])

print("✅ Labels encoded!")
print("Unique labels:", len(label_encoder.classes_))


✅ Labels encoded!
Unique labels: 243


In [7]:
print(label_encoder.classes_)


['aerial blight phytophthora' 'algal leaf spot green scurf'
 'alternaria leaf blight' 'alternaria leaf spot'
 'angular leaf spot bacterial' 'anthracnose' 'anthracnose colletotrichum'
 'apple scab' 'ascochyta blight' 'bacterial blast pseudomonas'
 'bacterial blight' 'bacterial blight angular leaf spot'
 'bacterial blight xanthomonas' 'bacterial canker' 'bacterial leaf blight'
 'bacterial leaf spot' 'bacterial ring rot' 'bacterial soft rot'
 'bacterial spot' 'bacterial wilt' 'basal rot' 'basal rot fusarium'
 'bayoud disease fusarium wilt' 'beech bark disease' 'berry disease'
 'bitter rot' 'black canker' 'black knot dibotryon'
 'black pod phytophthora' 'black rot' 'black rot bacterial'
 'black rot phytophthora' 'black scurf' 'black spot'
 'black spot alternaria' 'black spot diplocarpon' 'blackleg'
 'blast disease' 'blister blight' 'boll rot'
 'botryosphaeria panicle shoot blight' 'botrytis blight'
 'botrytis blight gray mold' 'bottom rot' 'boxwood blight'
 'bronze birch borer' 'brown blig

In [8]:
from transformers import DistilBertForSequenceClassification

num_labels = len(label_encoder.classes_)  # number of unique diseases

# Load pretrained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

print("✅ DistilBERT model loaded successfully!")
print("🧠 Ready to fine-tune for", num_labels, "plant diseases.")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ DistilBERT model loaded successfully!
🧠 Ready to fine-tune for 243 plant diseases.


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming your dataframe is still loaded as df
# (with 'Cleaned_Symptoms' and 'Cleaned_Disease_Name' columns)

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Cleaned_Disease_Name'])

# Split 80/10/10
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['Cleaned_Symptoms'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print("✅ Data split done successfully!")
print(f"Train size: {len(train_texts)} | Val size: {len(val_texts)} | Test size: {len(test_texts)}")


✅ Data split done successfully!
Train size: 2916 | Val size: 364 | Test size: 365


In [12]:
from transformers import DistilBertTokenizer

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

print("✅ Tokenization complete!")


✅ Tokenization complete!


In [14]:
from sklearn.preprocessing import LabelEncoder

# Encode disease labels
label_encoder = LabelEncoder()

train_labels_enc = label_encoder.fit_transform(train_labels)
val_labels_enc = label_encoder.transform(val_labels)
test_labels_enc = label_encoder.transform(test_labels)

print("✅ Labels encoded successfully!")
print(f"Total unique diseases: {len(label_encoder.classes_)}")


✅ Labels encoded successfully!
Total unique diseases: 243


In [15]:
import torch

class PlantDiseaseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create PyTorch dataset objects
train_dataset = PlantDiseaseDataset(train_encodings, train_labels_enc)
val_dataset = PlantDiseaseDataset(val_encodings, val_labels_enc)
test_dataset = PlantDiseaseDataset(test_encodings, test_labels_enc)

print("✅ Dataset objects created successfully!")


✅ Dataset objects created successfully!


# Set Up Training Configuration and Trainer

In [16]:
from transformers import Trainer, TrainingArguments

# ✅ Training configuration
training_args = TrainingArguments(
    output_dir="./results",                # folder to save model checkpoints
    learning_rate=2e-5,                    # how fast the model learns
    per_device_train_batch_size=8,         # batch size during training
    per_device_eval_batch_size=8,          # batch size during evaluation
    num_train_epochs=5,                    # try 5 for better learning
    weight_decay=0.01,                     # helps reduce overfitting
    logging_dir="./logs",                  # folder for logs
    save_total_limit=1                     # keep only the last checkpoint
)

# ✅ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

print("✅ Trainer setup complete! Ready to start training 🚀")


C:\Users\akhil\AppData\Local\Temp\ipykernel_15080\473394398.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✅ Trainer setup complete! Ready to start training 🚀


In [17]:
train_output = trainer.train()
print(train_output)


Step,Training Loss
500,4.957100
1000,3.697500
1500,2.854300


TrainOutput(global_step=1825, training_loss=3.6055207552975173, metrics={'train_runtime': 2796.3727, 'train_samples_per_second': 5.214, 'train_steps_per_second': 0.653, 'total_flos': 56826431069400.0, 'train_loss': 3.6055207552975173, 'epoch': 5.0})


In [18]:
from sklearn.metrics import accuracy_score, classification_report

# Get model predictions on validation data
preds_logits = trainer.predict(val_dataset).predictions
preds = preds_logits.argmax(axis=1)

# Compute metrics
accuracy = accuracy_score(val_labels_enc, preds)
print(f"✅ Validation Accuracy: {accuracy * 100:.2f}%")

# Detailed report
print("\n📊 Classification Report:")
print(classification_report(val_labels_enc, preds, zero_division=0))


✅ Validation Accuracy: 95.33%

📊 Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       0.50      1.00      0.67         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          16       1.00  

C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\akhil\OneDrive\Desk

In [19]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test set
test_preds_logits = trainer.predict(test_dataset).predictions
test_preds = test_preds_logits.argmax(axis=1)

# Compute test accuracy
test_accuracy = accuracy_score(test_labels_enc, test_preds)
print(f"🧪 Test Accuracy: {test_accuracy * 100:.2f}%")

# Detailed test report
print("\n📊 Test Classification Report:")
print(classification_report(test_labels_enc, test_preds, zero_division=0))


🧪 Test Accuracy: 95.62%

📊 Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           3       1.00      0.50      0.67         2
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00         1
           7       0.40      1.00      0.57         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         1
          16       1.00   

C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
C:\Users\akhil\OneDrive\Desktop\PlantDocBot\venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
C:\Users\akhil\OneDrive\Desk

In [20]:
# ✅ Create a folder to store model files
model.save_pretrained("../models/final_distilbert_model")
tokenizer.save_pretrained("../models/final_distilbert_model")

# ✅ Also save your label encoder
import joblib
joblib.dump(label_encoder, "../models/label_encoder.pkl")

print("✅ Model, tokenizer, and label encoder saved successfully!")


✅ Model, tokenizer, and label encoder saved successfully!


In [21]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import joblib

# ✅ Load everything
tokenizer = DistilBertTokenizer.from_pretrained("../models/final_distilbert_model")
model = DistilBertForSequenceClassification.from_pretrained("../models/final_distilbert_model")
label_encoder = joblib.load("../models/label_encoder.pkl")

print("✅ Model, tokenizer, and label encoder loaded successfully!")


✅ Model, tokenizer, and label encoder loaded successfully!


In [26]:
import joblib
label_encoder = joblib.load("../models/label_encoder.pkl")


In [28]:
import os

print(os.listdir("../models"))


['.ipynb_checkpoints', 'final_distilbert_model', 'final_plant_disease_NLP_model', 'label_encoder.pkl', 'logs', 'logs_top20', 'logs_top20_v2', 'mobilenetv2_final.h5', 'mobilenetv2_finetune.h5', 'mobilenetv2_stage1.h5', 'PlantDocBot_NLP_Model', 'PlantDocBot_Tokenizer', 'PlantDoc_TextModel_v2', 'PlantDoc_Text_Model', 'results', 'results_top20', 'results_top20_v2']


In [29]:
import joblib
label_encoder = joblib.load("../models/label_encoder.pkl")
print("✅ Label encoder loaded successfully!")
print("Number of classes:", len(label_encoder.classes_))


✅ Label encoder loaded successfully!
Number of classes: 243


In [32]:
from sklearn.preprocessing import LabelEncoder
import joblib

# recreate label encoder from the training dataframe
label_encoder = LabelEncoder()
label_encoder.fit(df["Cleaned_Disease_Name"])

# save it so we can reuse later
joblib.dump(label_encoder, "../models/label_encoder.pkl")

print("✅ Correct label encoder rebuilt and saved!")
print("Number of disease classes:", len(label_encoder.classes_))
print("Sample class names:", label_encoder.classes_[:15])


✅ Correct label encoder rebuilt and saved!
Number of disease classes: 243
Sample class names: ['aerial blight phytophthora' 'algal leaf spot green scurf'
 'alternaria leaf blight' 'alternaria leaf spot'
 'angular leaf spot bacterial' 'anthracnose' 'anthracnose colletotrichum'
 'apple scab' 'ascochyta blight' 'bacterial blast pseudomonas'
 'bacterial blight' 'bacterial blight angular leaf spot'
 'bacterial blight xanthomonas' 'bacterial canker' 'bacterial leaf blight']


In [33]:
import torch
import joblib

# Load encoder and test sample
label_encoder = joblib.load("../models/label_encoder.pkl")

text = "yellow circular spots on tomato leaves"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
outputs = model(**inputs)

probs = torch.nn.functional.softmax(outputs.logits, dim=1)
confidence = torch.max(probs).item() * 100
pred_label = torch.argmax(outputs.logits, dim=1).item()

pred_disease = label_encoder.inverse_transform([pred_label])[0]

print(f"🩺 Symptom: {text}")
print(f"🌿 Predicted Disease: {pred_disease}")
print(f"📊 Confidence: {confidence:.2f}%")


🩺 Symptom: yellow circular spots on tomato leaves
🌿 Predicted Disease: leaf spot common leaf spot
📊 Confidence: 2.28%


In [34]:
import numpy as np
import torch

# Get model predictions (logits) on validation set
val_logits = trainer.predict(val_dataset).predictions
val_probs = torch.nn.functional.softmax(torch.tensor(val_logits), dim=1).numpy()
val_preds = np.argmax(val_probs, axis=1)
val_true = np.array(val_labels_enc)

# Temperature scaling
def temperature_scale(logits, temp):
    logits = torch.tensor(logits)
    return torch.nn.functional.softmax(logits / temp, dim=1).numpy()

temps = np.linspace(0.1, 2.0, 20)
best_temp = 1.0
best_conf = 0

for t in temps:
    scaled_probs = temperature_scale(val_logits, t)
    mean_conf = np.mean(np.max(scaled_probs, axis=1))
    if mean_conf > best_conf:
        best_conf = mean_conf
        best_temp = t

print(f"🔥 Fitted temperature: {best_temp:.4f}, Mean confidence: {best_conf*100:.2f}%")


🔥 Fitted temperature: 0.1000, Mean confidence: 98.53%


In [35]:
import torch

temperature = 0.1  # from calibration step

texts = [
    "yellow circular spots on tomato leaves",
    "white powdery coating on leaves",
    "brown patches with dark edges on fruits"
]

for t in texts:
    inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)

    logits = outputs.logits / temperature  # 🔥 calibrated logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    confidence = torch.max(probs).item() * 100
    pred_label = torch.argmax(probs, dim=1).item()

    pred_disease = label_encoder.inverse_transform([pred_label])[0]

    print(f"🩺 Symptom: {t}")
    print(f"🌿 Predicted Disease: {pred_disease}")
    print(f"📊 Confidence: {confidence:.2f}%\n")


🩺 Symptom: yellow circular spots on tomato leaves
🌿 Predicted Disease: leaf spot common leaf spot
📊 Confidence: 64.19%

🩺 Symptom: white powdery coating on leaves
🌿 Predicted Disease: ganoderma butt rot
📊 Confidence: 70.10%

🩺 Symptom: brown patches with dark edges on fruits
🌿 Predicted Disease: fruit spot cercospora
📊 Confidence: 61.57%

